In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import os
import matplotlib.pyplot as plt
import shutil

IMG_WIDTH, IMG_HEIGHT = 150, 150
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 5
CLASS_NAMES = ['good', 'moderate', 'poor', 'unhealthy', 'severe']

def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def prepare_data(train_dir, validation_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    validation_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    return train_generator, validation_generator

def train_model(model, train_generator, validation_generator):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE
    )
    return history

def plot_accuracy_and_loss(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(EPOCHS)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

def predict_image(model, image_path):
    img = load_img(image_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = CLASS_NAMES[predicted_class_index]
    probability = np.max(predictions[0])

    return predicted_class_name, probability, predicted_class_index

def calculate_aqi(predicted_class_index, probability):
    aqi_ranges = {
        0: (0, 100),
        1: (100, 200),
        2: (200, 300),
        3: (300, 400),
        4: (400, 700)
    }

    if predicted_class_index not in aqi_ranges:
        return "Unknown class index"

    aqi_lower, aqi_higher = aqi_ranges[predicted_class_index]
    p = probability
    aqi = aqi_lower + (aqi_higher - aqi_lower) * (1 - p)

    return aqi

if __name__ == '__main__':
    print("Creating dummy dataset for demonstration purposes...")
    base_dir = './dell_project/dataset/pollution_image_dataset'
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'validation')

    for class_name in CLASS_NAMES:
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(validation_dir, class_name), exist_ok=True)
        for i in range(50):
            dummy_img = np.random.randint(0, 255, size=(IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
            plt.imsave(os.path.join(train_dir, class_name, f'train_img_{i}.png'), dummy_img)
        for i in range(10):
            dummy_img = np.random.randint(0, 255, size=(IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
            plt.imsave(os.path.join(validation_dir, class_name, f'val_img_{i}.png'), dummy_img)
    print("Dummy dataset created.")

    model = create_cnn_model((IMG_WIDTH, IMG_HEIGHT, 3), NUM_CLASSES)
    model.summary()

    train_generator, validation_generator = prepare_data(train_dir, validation_dir)

    print("\nStarting model training...")
    history = train_model(model, train_generator, validation_generator)
    print("Model training finished.")

    plot_accuracy_and_loss(history)
    
    final_val_accuracy = history.history['val_accuracy'][-1]
    print(f"\nFinal Validation Accuracy: {final_val_accuracy:.2%}")

    sample_image_path = os.path.join(validation_dir, 'severe', 'val_img_0.png')
    
    if os.path.exists(sample_image_path):
        print(f"\n--- Predicting for a sample image: {sample_image_path} ---")
        predicted_class, probability, class_index = predict_image(model, sample_image_path)
        
        print(f"Predicted Class: {predicted_class}")
        print(f"Confidence: {probability:.2%}")

        aqi_value = calculate_aqi(class_index, probability)
        print(f"Calculated AQI: {aqi_value:.2f}")
        
        img = load_img(sample_image_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
        plt.imshow(img)
        plt.title(f"Predicted: {predicted_class} ({probability:.2%})\nAQI: {aqi_value:.2f}")
        plt.axis('off')
        plt.show()
    else:
        print(f"\nSample image not found at: {sample_image_path}")
        print("Please provide a valid path to an image for prediction.")

    shutil.rmtree(base_dir)
    print(f"\nCleaned up dummy directory: {base_dir}")